In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# cd /content/gdrive/MyDrive/Colab Notebooks/RBM - CF/Recommender-System

In [1]:
from collaborative_filtering import CF 
from demographic_filtering import DF
from perceptron import Perceptron as pct
from get_data import (
    convert,
    get_users_data,
    get_items_data,
    get_rating_base_data,
    get_rating_test_data,
    get_rating_data
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.autograd import Variable
# Import RBM model script
from rbm_torch import RBM

In [ ]:
# # Initialize Weights and Biases
# import wandb
# wandb.init(entity="phusdt", project="boltzmann_machines_collaborative_filtering")

# # Config is a variable that holds and saves hyper-parameters
# config = wandb.config  # Initialize config
# config

In [45]:
# CF 
RATE_TRAIN = get_rating_base_data().values # convert to matrix
RATE_TEST = get_rating_test_data().values # convert to matrix

RATE_TRAIN[:, :2] -= 1 # start from 0
RATE_TEST[:, :2] -= 1

CF = CF(RATE_TRAIN, k=25)
CF.fit()

print('Similar Matrix Works')
print(CF.S)
print('Number of rows: ', CF.S.shape[0])
print('Number of columns: ', CF.S.shape[1])

ids = np.where(RATE_TEST[:, 0] == 0)[0].astype('int32')
real_items_1 = RATE_TEST[(np.where((RATE_TEST[:, 0] == 0) & (RATE_TEST[:, 2] >= 3)))]
predicted_items = []

for row in RATE_TEST[ids, :]:
    predicted_rating = CF.pred(0, row[1])
    if predicted_rating >= 3:
        predicted_items.append(row[1])

print('Items which user 1 actually like: ', real_items_1[:, 1])
print('Items in prediction which user 1 might like ', predicted_items)


n_test = RATE_TEST.shape[0]
correct_items_count = 0
real_items_user_like_count = len(np.where(RATE_TEST[:, 2] >= 3)[0].astype(np.int32))

user_id = 0
while user_id < CF.n_users:
    ids = np.where(RATE_TEST[:, 0] == user_id)[0].astype('int32')
    real_items = RATE_TEST[(np.where((RATE_TEST[:, 0] == user_id) & (RATE_TEST[:, 2] >= 3)))]
    for row in RATE_TEST[ids, :]:
        predicted_rating = CF.pred(user_id, row[1])
        if predicted_rating >= 3 and row[1] in real_items:
            correct_items_count += 1
    user_id += 1
accuracy = correct_items_count/real_items_user_like_count
print('The accuracy of Collaborative Filtering: {0}/{1} = {2}'.format(correct_items_count, real_items_user_like_count, accuracy))

Similar Matrix Works
[[ 1.   -0.08  0.01 ... -0.08  0.03 -0.06]
 [-0.08  1.    0.05 ... -0.11  0.    0.22]
 [ 0.01  0.05  1.   ... -0.01 -0.04  0.05]
 ...
 [-0.08 -0.11 -0.01 ...  1.    0.   -0.18]
 [ 0.03  0.   -0.04 ...  0.    1.    0.  ]
 [-0.06  0.22  0.05 ... -0.18  0.    1.  ]]
Number of rows:  943
Number of columns:  943
Items which user 1 actually like:  [  0   1   8  21  25  29  31  37  39  67  74  88  92  98 118 132 134 135
 140 143 145 146 157 165 172 175 178 180 186 193 196 204 210 219 233 238
 245 248 255 256 267 268 269]
Items in prediction which user 1 might like  [0, 1, 7, 8, 20, 21, 25, 29, 31, 33, 36, 37, 39, 62, 67, 74, 88, 92, 98, 104, 118, 130, 132, 134, 135, 137, 140, 143, 145, 146, 157, 165, 172, 175, 178, 180, 186, 193, 196, 204, 210, 219, 233, 236, 238, 245, 248, 255, 256, 267, 268, 269, 270]
The accuracy of Collaborative Filtering: 14963/16558 = 0.903671941055683


In [ ]:
from sklearn.metrics import recall_score, precision_score
x = list(set(get_rating_base_data().iloc[:,1]))
x = pd.DataFrame(columns=x)
# lấy dữ liệu của những iteam thực sự được người dùng thích
zipper = np.ones(len(real_items_1[:, 1]))
dict_ = dict(zip(real_items_1[:, 1], zipper))
y_real = x.append(dict_,ignore_index=True).fillna(0).values

# lấy dữ liệu của những item predict
zipper = np.ones(len(predicted_items))
dict_ = dict(zip(predicted_items, zipper))
y_pred = x.append(dict_,ignore_index=True).fillna(0).values

recall = recall_score(y_real, y_pred, average='macro')
precision = precision_score(y_real, y_pred, average='macro')

print('Chỉ số recall: {}\nChỉ số precision: {}'.format(recall, precision))

In [20]:
# DF
USERS = get_users_data("ml-100k", "u.user")
RATE_TRAIN = get_rating_data("ml-100k", "u2.base").values
RATE_TEST = get_rating_data("ml-100k", "u2.test").values

RATE_TRAIN[:, :2] -= 1 # start from 0
RATE_TEST[:, :2] -= 1
DF = DF(USERS, RATE_TRAIN, 25)
DF.fit()

print("Ma trận tương đồng thông tin")
print(DF.similarities)
print("Số hàng của ma trận:", DF.similarities.shape[0])
print("Số cột của ma trận: ", DF.similarities.shape[1])

ids = np.where(RATE_TEST[:, 0] == 0)[0].astype("int32")
real_items_1 = RATE_TEST[(np.where((RATE_TEST[:, 0] == 0) & (RATE_TEST[:, 2] >= 3)))]
correct_predicted_items = []

for row in RATE_TEST[ids, :]:
    predicted_rating = DF.pred(0, row[1])
    if predicted_rating >= 3 and row[1] in real_items_1:
        correct_predicted_items.append(row[1])


print("Những items user 1 thật sự thích       : ", real_items_1[:, 1])
print("Những items user 1 được dự đoán thích  : ", correct_predicted_items)

n_test = RATE_TEST.shape[0]
correct_items_count = 0
real_items_user_like_count = len(np.where(RATE_TEST[:, 2] >= 3)[0].astype(np.int32))

user_id = 0
while user_id < DF.n_users:
    ids = np.where(RATE_TEST[:, 0] == user_id)[0].astype("int32")
    real_items = RATE_TEST[(np.where((RATE_TEST[:, 0] == user_id) & (RATE_TEST[:, 2] >= 3)))]
    for row in RATE_TEST[ids, :]:
        predicted_rating = DF.pred(user_id, row[1])
        if predicted_rating >= 3 and row[1] in real_items:
            correct_items_count = correct_items_count + 1
    user_id = user_id + 1

print("Độ chính xác của Demographic Filtering :", correct_items_count / real_items_user_like_count)

Ma trận tương đồng thông tin
[[1.         0.         0.66666667 ... 0.66666667 0.         0.66666667]
 [0.         1.         0.         ... 0.         0.33333333 0.        ]
 [0.66666667 0.         1.         ... 0.66666667 0.         0.66666667]
 ...
 [0.66666667 0.         0.66666667 ... 1.         0.         1.        ]
 [0.         0.33333333 0.         ... 0.         1.         0.        ]
 [0.66666667 0.         0.66666667 ... 1.         0.         1.        ]]
Số hàng của ma trận: 943
Số cột của ma trận:  943
Những items user 1 thật sự thích       :  [  0   1   8  21  25  29  31  37  39  67  74  88  92  98 118 132 134 135
 140 143 145 146 157 165 172 175 178 180 186 193 196 204 210 219 233 238
 245 248 255 256 267 268 269]
Những items user 1 được dự đoán thích  :  [0, 1, 8, 21, 25, 29, 31, 37, 67, 74, 88, 92, 98, 118, 132, 134, 135, 140, 143, 146, 165, 172, 175, 178, 180, 186, 193, 196, 204, 210, 219, 233, 238, 245, 248, 255, 256, 267, 268, 269]
Độ chính xác của Demographic Fil

In [21]:
# predict for all user from test set
recommended_all_users = DF.recommend_all_users(RATE_TEST)

In [22]:
# create a dataframe to store all value
recommended_all_users = recommended_all_users.astype(int)
df_test = pd.DataFrame(recommended_all_users, columns = ['user_id', 'movie_id', 'rating'])
# matrix index from 0 -> 1
df_test['user_id'] = df_test['user_id'] + 1
df_test['movie_id'] = df_test['movie_id'] + 1
# pivot table 
df_test = pd.pivot_table(df_test, values='rating', index=['user_id'], 
                         columns=['movie_id'], aggfunc=np.mean).fillna(0)
df_test

movie_id,1,2,3,4,5,6,7,8,9,10,...,1642,1643,1645,1646,1650,1651,1653,1656,1662,1664
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,4.0,4.0,0.0,4.0
2,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,...,5.0,4.0,4.0,0.0,4.0,4.0,5.0,0.0,5.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,3.0,3.0,0.0,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,0.0,5.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,3.0,0.0,3.0,3.0,4.0,0.0,0.0,0.0
6,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,5.0,0.0,0.0,4.0,0.0,0.0,5.0,5.0,0.0,0.0
8,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,0.0,0.0


In [23]:
y_true =get_rating_data("ml-100k", "u2.test")
y_true = pd.pivot_table(y_true, values='rating', index=['user_id'],
                    columns=['movie_id'], aggfunc=np.mean).fillna(0)
y_true

movie_id,1,2,3,4,5,6,7,8,9,10,...,1603,1605,1608,1615,1616,1617,1619,1620,1623,1624
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# # Perceptron
# ids = np.where(RATE_TEST[:, 0] == 0)[0].astype("int32")

# MATRIX_DF = []
# MATRIX_CF = []

# for row in RATE_TEST[ids, :]:
#     p_cf = CF.pred(0, row[1])
#     p_df = DF.pred(u=0, i=row[2])
#     MATRIX_CF.append([0, row[1], p_cf])
#     MATRIX_DF.append([0, row[1], p_df])
# MATRIX_CF = np.asarray(MATRIX_CF)
# MATRIX_DF = np.asarray(MATRIX_DF)

# CF_predicted = np.asanyarray(MATRIX_CF[:, 2])
# DF_predicted = MATRIX_DF[:, 2]
# true_rating = RATE_TEST[ids, 2]

# dataset = np.c_[CF_predicted, DF_predicted, true_rating]

# # print("Ma trận dự đoán đánh giá CF, DF, True Rating")
# # print(dataset)

# PLA = Perceptron(dataset, 0.003, len(ids))
# PLA.fit()
# predicted_ratings_pla = PLA.predict()

# print("Dự đoán đánh giá sau khi được điều chỉnh")
# print(np.round(predicted_ratings_pla, 3))


# Restricted Boltzmann Machines

### Get dataset 
<h5><b> With dataset as output of model demographic filtering</b></h5>

In [5]:
ratings = get_rating_base_data()
df_train = pd.pivot_table(ratings, values='rating', index=['user_id'],
                    columns=['movie_id'], aggfunc=np.mean).fillna(0)
df_train

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,3.0,3.0,5.0,4.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Train and valid split
- 90% data to train
- 10% data to validate

In [6]:
# Split the ratings data frame into 90% training and 10% to validate
train, valid = train_test_split(ratings, train_size=0.9)

# Convert training and test data into Numpy arrays
trainingset = np.array(train, dtype='int')
valid = np.array(valid, dtype='int')

# Collect the total number of movies and users in order to then make a matrix of the data
nb_users = int(max(max(trainingset[:, 0]), max(valid[:, 0])))
nb_movies = int(max(max(trainingset[:, 1]), max(valid[:, 1])))

### Convert data into Pytorch Tensors and binary rating

In [7]:
# Convert training and test sets into arrays
training_set = convert(trainingset, nb_users, nb_movies)
val_set = convert(valid, nb_users, nb_movies)

### Parameters Initialize

In [8]:
# Number of users is the number of visible units
n_vis = len(training_set[0])

# This tunable parameter is the number of features that we want to detect (number of hidden units)
n_hid = 100

# Number of users
nb_users = DF.n_users

# Number of items
nb_items = DF.n_items

# set batch size to be 512 (tunable)
batch_size_ = 512

# run for 50 epoch
nb_epoch = 200  

### Training RBM model

In [9]:
# Create the model object RBM()
rbm = RBM(n_vis=n_vis, n_hid=n_hid, nb_epoch=nb_epoch, nb_users=nb_users, batch_size_=batch_size_)
rbm.train(training_set)

Epoch: 1- RMSE Reconstruction Error: 0.6870768070220947
Epoch: 2- RMSE Reconstruction Error: 0.7529663443565369
Epoch: 3- RMSE Reconstruction Error: 0.633105993270874
Epoch: 4- RMSE Reconstruction Error: 0.5240163803100586
Epoch: 5- RMSE Reconstruction Error: 0.48365774750709534
Epoch: 6- RMSE Reconstruction Error: 0.5135076642036438
Epoch: 7- RMSE Reconstruction Error: 0.48506465554237366
Epoch: 8- RMSE Reconstruction Error: 0.6361340284347534
Epoch: 9- RMSE Reconstruction Error: 0.47793176770210266
Epoch: 10- RMSE Reconstruction Error: 0.8035410642623901
Epoch: 11- RMSE Reconstruction Error: 0.5082522630691528
Epoch: 12- RMSE Reconstruction Error: 0.7870728969573975
Epoch: 13- RMSE Reconstruction Error: 0.5089757442474365
Epoch: 14- RMSE Reconstruction Error: 0.7785593867301941
Epoch: 15- RMSE Reconstruction Error: 0.5070978999137878
Epoch: 16- RMSE Reconstruction Error: 0.771454930305481
Epoch: 17- RMSE Reconstruction Error: 0.4957364499568939
Epoch: 18- RMSE Reconstruction Error: 0

Epoch: 145- RMSE Reconstruction Error: 0.5173417329788208
Epoch: 146- RMSE Reconstruction Error: 0.8021661043167114
Epoch: 147- RMSE Reconstruction Error: 0.5159172415733337
Epoch: 148- RMSE Reconstruction Error: 0.8018432259559631
Epoch: 149- RMSE Reconstruction Error: 0.5152563452720642
Epoch: 150- RMSE Reconstruction Error: 0.8027265667915344
Epoch: 151- RMSE Reconstruction Error: 0.5162869095802307
Epoch: 152- RMSE Reconstruction Error: 0.8016562461853027
Epoch: 153- RMSE Reconstruction Error: 0.5162076950073242
Epoch: 154- RMSE Reconstruction Error: 0.8027435541152954
Epoch: 155- RMSE Reconstruction Error: 0.5159172415733337
Epoch: 156- RMSE Reconstruction Error: 0.8027265667915344
Epoch: 157- RMSE Reconstruction Error: 0.5162076950073242
Epoch: 158- RMSE Reconstruction Error: 0.8019621968269348
Epoch: 159- RMSE Reconstruction Error: 0.5154415369033813
Epoch: 160- RMSE Reconstruction Error: 0.8018772006034851
Epoch: 161- RMSE Reconstruction Error: 0.5156794190406799
Epoch: 162- RM

### Validation RBM model

In [10]:
val_loss = 0
rmse_val = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = val_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        val_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        rmse_val += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2))
        s += 1.
print('Valid Loss: '+str(val_loss/s))
print('RMSE: '+str(rmse_val))

Valid Loss: tensor(0.3163)
RMSE: tensor(432.2324)


### Extracting the title of the Movies from the Movies dataset

In [25]:
movies = get_items_data()
movie_title = movies.iloc[:nb_movies, 1:2]
movie_title = pd.DataFrame.transpose(movie_title)
movie_title

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
movie_title,Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Twelve Monkeys (1995),Babe (1995),Dead Man Walking (1995),Richard III (1995),...,Mirage (1995),Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998),You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)


In [43]:
df_empty = pd.DataFrame(columns=[str(col) for col in range(1, DF.n_items + 1)],
                        index=[str(index) for index in range(1, len(df_test) + 1)])


1482

In [ ]:
for user in range(1, df_test.shape[0]):
    for item in range(1, df_test.shape[1]):
        df_empty.iloc[user,item] = df_test.iloc[user,item]
df_empty

### Choosing a User by the Test Set and taking the whole list of Movies of that User and convert it to a PyTorch Tensor

In [11]:
user_id = 101
user_input = Variable(val_set[user_id - 1]).unsqueeze(0)

### Getting the Recommendations

In [13]:
# predict
output = rbm.predict(user_input)
output = output.data.numpy()


<h1>Version 2</h1>

In [37]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
from dataset import _get_training_data, _get_test_data
from rbm_model import RBM
import numpy as np
import sys
sys.argv = sys.argv[:1]

In [38]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
del_all_flags(tf.flags.FLAGS)

tf.app.flags.DEFINE_string('tf_records_train_path', '/Users/phusdt/GitHub/Recommender-System/tf_records_100k/train/',
                           'Path of the training data.')

tf.app.flags.DEFINE_string('tf_records_test_path', '/Users/phusdt/GitHub/Recommender-System/tf_records_100k/test/',
                           'Path of the test data.')

tf.app.flags.DEFINE_integer('num_epoch', 100,
                            'Number of training epochs.')

tf.app.flags.DEFINE_integer('batch_size', 32,
                            'Size of the training batch.')

tf.app.flags.DEFINE_float('learning_rate',1.0,
                          'Learning_Rate')

tf.app.flags.DEFINE_integer('num_v', 1682,
                            'Number of visible neurons (Number of movies the users rated.)')

tf.app.flags.DEFINE_integer('num_h', 10,
                            'Number of hidden neurons.')

tf.app.flags.DEFINE_integer('num_samples', 943,
                            'Number of training samples (Number of users, who gave a rating).')

tf.app.flags.DEFINE_integer('k', 10,
                           'Number of iterations during gibbs samling.')

tf.app.flags.DEFINE_integer('eval_after',50,
                            'Evaluate model after number of iterations.')

FLAGS = tf.app.flags.FLAGS

In [39]:
train_data, train_data_infer=_get_training_data(FLAGS)

In [40]:
'''Building the graph, opening of a session and starting the training od the neural network.'''
    
num_batches=int(FLAGS.num_samples/FLAGS.batch_size)

train_data, train_data_infer=_get_training_data(FLAGS)
test_data=_get_test_data(FLAGS)

iter_train = tf.data.make_initializable_iterator(train_data)
iter_train_infer = tf.data.make_initializable_iterator(train_data_infer)
iter_test = tf.data.make_initializable_iterator(test_data)

x_train= iter_train.get_next()
x_train_infer=iter_train_infer.get_next()
x_test=iter_test.get_next()

model=RBM(FLAGS)

update_op, accuracy=model.optimize(x_train)
v_infer=model.inference(x_train_infer)

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    for epoch in range(FLAGS.num_epoch):

        acc_train=0
        acc_infer=0

        sess.run(iter_train.initializer)

        for batch_nr in range(num_batches):
            _, acc=sess.run((update_op, accuracy))
            acc_train+=acc

            if batch_nr>0 and batch_nr%FLAGS.eval_after==0:

                sess.run(iter_train_infer.initializer)
                sess.run(iter_test.initializer)

                num_valid_batches=0

                for i in range(FLAGS.num_samples):

                    v_target=sess.run(x_test)[0]

                    if len(v_target[v_target>=0])>0:

                        v_=sess.run(v_infer)[0]
                        acc=1.0-np.mean(np.abs(v_[v_target>=0]-v_target[v_target>=0]))
                        acc_infer+=acc
                        num_valid_batches+=1

                print('epoch_nr: %i, batch: %i/%i, acc_train: %.3f, acc_test: %.3f'%
                      (epoch, batch_nr, num_batches, (acc_train/FLAGS.eval_after), (acc_infer/num_valid_batches)))

                acc_train=0
                acc_infer=0

RuntimeError: dataset.make_initializable_iterator is not supported when eager execution is enabled. Use `for element in dataset` instead.